<a href="https://colab.research.google.com/github/arturomolin/Tarea1/blob/main/Ejercicio_26.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Ejercicio 26
import numpy as np
import matplotlib.pyplot as plt

# Esta función genera la matriz de Hilbert de tamaño n x n, cada entrada hij está dada por 1 / (i + j + 1) porque Python empieza desde 0
def hilbert_matrix(n):
    return np.array([[1 / (i + j + 1) for j in range(n)] for i in range(n)])

# Gram-Schmidt clásico: vamos sacando los vectores ortogonales uno por uno
def gram_schmidt_classic(A):
    n = A.shape[1]
    Q = np.zeros_like(A)
    for j in range(n):
        q = A[:, j]
        for i in range(j):
            q -= np.dot(Q[:, i], A[:, j]) * Q[:, i]
        Q[:, j] = q / np.linalg.norm(q)
    return Q

# Gram-Schmidt modificado: parecida pero se hace diferente para que sea más estable
def gram_schmidt_modified(A):
    n = A.shape[1]
    Q = np.zeros_like(A)
    for j in range(n):
        q = A[:, j]
        for i in range(j):
            r = np.dot(Q[:, i], q)
            q -= r * Q[:, i]
        Q[:, j] = q / np.linalg.norm(q)
    return Q

# Aquí aplicamos Gram-Schmidt clásico dos veces seguidas, usando la Q resultante como entrada
def gram_schmidt_twice(A):
    Q1 = gram_schmidt_classic(A)
    Q2 = gram_schmidt_classic(Q1)
    return Q2

# Factorización de Householder: va eliminando abajo de la diagonal con reflexiones
def householder_qr(A):
    m, n = A.shape
    Q = np.eye(m)
    for k in range(n):
        x = A[k:, k]
        e1 = np.zeros_like(x)
        e1[0] = 1
        # creamos el vector.
        v = x + np.sign(x[0]) * np.linalg.norm(x) * e1
        v /= np.linalg.norm(v)
        Hk = np.eye(m)
        Hk[k:, k:] -= 2.0 * np.outer(v, v)  # matriz de Householder
        A = Hk @ A
        Q = Q @ Hk
    return Q

# Cholesky: usamos las ecuaciones normales AtA = LLᵗ y sacamos Q = A L^{-T}
def cholesky_q(A):
    AtA = A.T @ A
    L = np.linalg.cholesky(AtA)
    Q = A @ np.linalg.inv(L.T)
    return Q

# Esta función calcula la pérdida de ortogonalidad como dice el problema: -log10(||I - QᵗQ||)
def loss_of_orthogonality(Q):
    I = np.eye(Q.shape[1])
    diff = I - Q.T @ Q
    return -np.log10(np.linalg.norm(diff))

# Ahora recorremos varios tamaños de n desde 2 hasta 12
n_values = range(2, 13)
classic_losses = []
modified_losses = []
twice_losses = []
householder_losses = []
cholesky_losses = []

# Para cada n, generamos la matriz de Hilbert y aplicamos todos los métodos
for n in n_values:
    H = hilbert_matrix(n)

    Q_classic = gram_schmidt_classic(H)
    Q_modified = gram_schmidt_modified(H)
    Q_twice = gram_schmidt_twice(H)
    Q_householder = householder_qr(H)
    Q_cholesky = cholesky_q(H)

    # Guardamos las pérdidas de ortogonalidad para graficar después
    classic_losses.append(loss_of_orthogonality(Q_classic))
    modified_losses.append(loss_of_orthogonality(Q_modified))
    twice_losses.append(loss_of_orthogonality(Q_twice))
    householder_losses.append(loss_of_orthogonality(Q_householder))
    cholesky_losses.append(loss_of_orthogonality(Q_cholesky))

# Finalmente, graficamos todo para comparar los métodos
plt.figure(figsize=(10, 6))
plt.plot(n_values, classic_losses, label='Gram-Schmidt Clásico')
plt.plot(n_values, modified_losses, label='Gram-Schmidt Modificado')
plt.plot(n_values, twice_losses, label='Gram-Schmidt Doble')
plt.plot(n_values, householder_losses, label='Householder')
plt.plot(n_values, cholesky_losses, label='Cholesky')
plt.xlabel('n (tamaño de la matriz de Hilbert)')
plt.ylabel('-log10(||I - QᵀQ||)')
plt.title('Pérdida de Ortogonalidad de Diferentes Métodos')
plt.legend()
plt.grid(True)
plt.show()
